<a href="https://colab.research.google.com/github/alonzojp/Magnolia-Beach-Model/blob/main/Functionized_Magnolia_Beach_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Imports
import pandas as pd
import plotly.graph_objects as go
import math
import plotly.express as px
import tensorflow as tf
import random
from tensorflow import keras
import time

import warnings
warnings.filterwarnings("ignore")

### **Ports Dataframe**

In [2]:
# Combine Port Lavaca & OConnor data together
def generate_ports_df(LAVACA_DATA_PATH, OCONNOR_DATA_PATH):
  lavaca_df = pd.read_csv(LAVACA_DATA_PATH)[:-11]
  oconnor_df = pd.read_csv(OCONNOR_DATA_PATH)[:-11]
  ports_df = pd.concat([lavaca_df, oconnor_df], axis=1)

  ports_df.index = lavaca_df['#date+time']
  ports_df.drop(columns=['#date+time'], inplace=True)
  ports_df.index = pd.to_datetime(ports_df.index, format='%m-%d-%Y %H:%M')

  ports_df[ports_df.columns] = ports_df[ports_df.columns].apply(pd.to_numeric, errors='coerce')

  ports_df = ports_df.drop(columns = ['057-wsd', '057-wdr'])
  ports_df.drop(ports_df['2016-10-30':'2016-11-08'].index, inplace=True) # 033-wsd error

  return ports_df

In [3]:
# Visualization of all data in dataset
def graph_data(df):
  for column in df.columns:
    fig = go.Figure(data=go.Scatter(x=df.index, y=df[column], mode='lines', line_color='#003f5c', name = ""))
    fig.update_traces(connectgaps=False)
    fig.update_layout(title = column, xaxis_title="", yaxis_title="")
    fig.show()

In [4]:
# Converts wsd and wdr to X and Y wind data
def convert_winds(ports_df):
  ports = ['033']
  for port in ports:
    x_wind = []
    y_wind = []
    for i in range(0, len(ports_df)):
      radius = ports_df[port + '-wsd'][i]
      theta = ports_df[port + '-wdr'][i]
      theta = theta * (math.pi / 180)
      x_wind.append(radius * math.cos(theta))
      y_wind.append(radius * math.sin(theta))
    ports_df[port + "-Xwind"] = x_wind
    ports_df[port + "-Ywind"] = y_wind
    del ports_df[port + '-wsd']
    del ports_df[port + '-wdr']
  return ports_df

In [5]:
# Creates estimated Magnolia data
def create_magnolia_data(ports_df):
  ports_df['mag-pwl'] = (ports_df['033-pwl'] + ports_df['057-pwl']) / 2
  ports_df['mag-surge'] = (ports_df['033-surge'] + ports_df['057-surge']) / 2
  ports_df['mag-harmwl'] = (ports_df['033-harmwl'] + ports_df['057-harmwl']) / 2
  return ports_df

In [6]:
LAVACA_DATA_PATH = '/content/Lavaca Data 09_24.csv' #033
OCONNOR_DATA_PATH = '/content/OConnor Data 09_24.csv' #057

ports_df = generate_ports_df(LAVACA_DATA_PATH, OCONNOR_DATA_PATH)
ports_df = create_magnolia_data(ports_df)
ports_df = convert_winds(ports_df)
# graph_data(ports_df)
ports_df

,033-pwl,033-harmwl,033-surge,057-pwl,057-harmwl,057-surge,mag-pwl,mag-surge,mag-harmwl,033-Xwind,033-Ywind
#date+time,,,,,,,,,,,
2010-01-01 00:00:00,0.070,-0.149,0.219,0.153,0.032,0.121,0.1115,0.1700,-0.0585,NaN,NaN
2010-01-01 01:00:00,0.124,-0.083,0.207,0.191,0.073,0.118,0.1575,0.1625,-0.0050,NaN,NaN
2010-01-01 02:00:00,0.182,-0.017,0.199,0.216,0.108,0.108,0.1990,0.1535,0.0455,NaN,NaN
2010-01-01 03:00:00,0.224,0.042,0.182,0.292,0.133,0.159,0.2580,0.1705,0.0875,NaN,NaN
2010-01-01 04:00:00,0.202,0.093,0.109,0.267,0.148,0.119,0.2345,0.1140,0.1205,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2025-01-01 19:00:00,NaN,-0.231,NaN,NaN,-0.207,NaN,NaN,NaN,-0.2190,NaN,NaN
2025-01-01 20:00:00,NaN,-0.272,NaN,NaN,-0.195,NaN,NaN,NaN,-0.2335,NaN,NaN
2025-01-01 21:00:00,NaN,-0.295,NaN,NaN,-0.169,NaN,NaN,NaN,-0.2320,NaN,NaN


### **ANN #1**

In [7]:
# Generates first ANN dataframe
def create_first_ann_df(ports_df, LEAD_TIME):
  ports_df['TARGET'] = ports_df['mag-surge'].shift(-1 * LEAD_TIME + 5) # accounts for 5hr delay
  ports_df['REMOVE (for later conversion)'] = ports_df['mag-harmwl'].shift(-1 * LEAD_TIME + 5) # used later for conversion
  ports_df = ports_df[['033-pwl',	'033-surge',	'033-Xwind',	'033-Ywind', '057-pwl',	'057-surge', 'mag-pwl', 'mag-surge', 'TARGET', 'REMOVE (for later conversion)']]

  organized_df = pd.DataFrame() # adds 'hours back' features
  column_names = ['033-pwl',	'033-surge',	'057-pwl',	'057-surge', 'mag-pwl', 'mag-surge', '033-Xwind',	'033-Ywind']
  for column in column_names:
    for i in range(24, 0, -1):
      df_shifted = ports_df.shift(1 * i)
      organized_df[column + " -" + str(i)] = df_shifted[column]
    organized_df[column] = ports_df[column]

  first_winds_df = pd.DataFrame() # first ANN's wind perfect prog
  for i in range(1, LEAD_TIME + 1 - 5):
    df_reverse_shifted = ports_df.shift(-1 * i)
    first_winds_df['033-Xwind' + " +" + str(i)] = df_reverse_shifted['033-Xwind']
  for i in range(1, LEAD_TIME + 1 - 5):
    df_reverse_shifted = ports_df.shift(-1 * i)
    first_winds_df['033-Ywind' + " +" + str(i)] = df_reverse_shifted['033-Ywind']

  second_winds_df = pd.DataFrame() # second ANN's wind perfect prog
  for i in range(1, LEAD_TIME + 1):
    df_reverse_shifted = ports_df.shift(-1 * i)
    second_winds_df['033-Xwind' + " +" + str(i)] = df_reverse_shifted['033-Xwind']
  for i in range(1, LEAD_TIME + 1):
    df_reverse_shifted = ports_df.shift(-1 * i)
    second_winds_df['033-Ywind' + " +" + str(i)] = df_reverse_shifted['033-Ywind']

  first_df = pd.concat([organized_df, first_winds_df], axis=1) # creating first dataset
  first_df['TARGET'] = ports_df['TARGET']
  first_df['REMOVE (for later conversion)'] = ports_df['REMOVE (for later conversion)']
  first_df = first_df.dropna()
  first_pwl_conversion = first_df['REMOVE (for later conversion)']
  del first_df['REMOVE (for later conversion)']
  return first_df, first_pwl_conversion, second_winds_df

In [8]:
# Generates first ANN data split
def generate_first_ann_split(first_df):
  length = len(first_df)
  train_length = round(length * .6)
  val_length = round(length * .8)

  train_df = first_df[:train_length]
  val_df = first_df[train_length:val_length]
  test_df = first_df[val_length:]

  X_train = train_df.drop('TARGET',axis=1)
  y_train = train_df['TARGET']
  X_val = val_df.drop('TARGET',axis=1)
  y_val = val_df['TARGET']
  X_test = test_df.drop('TARGET',axis=1)
  y_test = test_df['TARGET']

  return X_train, y_train, X_val, y_val, X_test, y_test

In [9]:
# Plots train, val, and test distribution
def plot_first_ann(y_train, y_val, y_test):
  fig_train = px.line(y_train)
  fig_train.update_traces(line_color='#EF2648')
  fig_train.update_traces(name='Training')

  fig_val = px.line(y_val)
  fig_val.update_traces(line_color='#FBC800')
  fig_val.update_traces(name='Validation')

  fig_test = px.line(y_test)
  fig_test.update_traces(line_color='#5A8A91')
  fig_test.update_traces(name='Testing')

  fig = go.Figure(data = fig_train.data + fig_val.data + fig_test.data)
  fig.show()

In [10]:
# Trains first ANN
def train_first_ann(X_train, y_train, X_val, y_val, X_test, y_test, units1, activation1):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  patience=20,
                                                  mode='min')

  model = tf.keras.Sequential([
      tf.keras.layers.Dense(units1,
                            activation= activation1,
                            input_shape = [X_train.shape[1]],
                            kernel_regularizer = tf.keras.regularizers.L2(l2=0.0001)
                            ),
      tf.keras.layers.Dense(1),
  ])

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

  losses = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stopping],
                    epochs=1000, batch_size = 32,
                    shuffle=True, verbose = 0)

  return model, losses

In [11]:
# Plots training loss curve
def plot_loss_curve(losses):
  loss_df = pd.DataFrame(losses.history)
  loss_df.loc[:,['loss','val_loss']].plot()

In [12]:
# Generate first ANN predictions
def generate_first_ann_predictions(model, X_test, y_test):
  predictions = model.predict(X_test, verbose = 0)

  predictions_df = pd.DataFrame()
  predictions_df.index = X_test.index
  predictions_df['Predictions'] = predictions
  predictions_df['Actual'] = y_test

  predictions_df = predictions_df.resample('60min').mean()
  return predictions_df

In [13]:
# Converts predictted surge to primary water level
def convert_surge_to_pwl(predictions_df, pwl_conversion):
  filtered_conversion_df = pwl_conversion[predictions_df.index[0]:predictions_df.index[-1]]

  predictions_df['PredictedPWL'] = predictions_df['Predictions'] + filtered_conversion_df
  predictions_df['ActualPWL'] = predictions_df['Actual'] + filtered_conversion_df
  return predictions_df

In [14]:
# Plot predictions against actual for 1st ANN
def plot_first_ann_predictions(predictions_df):
  fig = go.Figure(data=go.Scatter(x=predictions_df.index, y=predictions_df['ActualPWL'], mode='lines', line_color='#000000', name = "Actual"))
  fig.add_trace(go.Scatter(x=predictions_df.index, y=predictions_df['PredictedPWL'],mode='lines', line=dict(color="#FF0000"), name = "Predictions"))
  fig.update_traces(connectgaps=False)
  predictions_df = predictions_df.dropna()
  fig.show()

In [15]:
# Produces evaluations for the first ANN
def evaluate_first_ann(predictions_df):
  temp = predictions_df.copy().dropna()
  mae = sum(abs(temp['PredictedPWL'] - temp['ActualPWL'])) / len(temp['PredictedPWL'])
  mae = round(mae * 100, 2)

  counter = 0
  array = abs(temp['PredictedPWL'] - temp['ActualPWL'])
  # array = np.array(array)
  for i in range(0, len(array)):
    if(array[i] > .15):
      counter += 1

  cf = (float(f'{100- (counter / len(array) * 100):.2f}'))
  return mae, cf

### **Magnolia Beach**

In [16]:
# Creates magnolia dataframe with correct timestamps
def create_magnolia_df(MAGNOLIA_DATA_PATH, CHANGING_TIME):
  magnolia_df = pd.read_csv(MAGNOLIA_DATA_PATH, index_col = 0)
  magnolia_df = magnolia_df.dropna() # remove data inconsistency
  magnolia_df.index = pd.to_datetime(magnolia_df.index, format='mixed', utc=True) # Converting index to DateTimeIndex
  magnolia_df.index = pd.to_datetime(magnolia_df.index, format='%Y-%m-%d %H:%M') # converts index to date
  magnolia_df.index = magnolia_df.index.tz_localize(None)
  magnolia_df.index.names = ['Timestamp (UTC)']
  magnolia_df['Distance to Water Measurement (m)'] = magnolia_df['Distance to Water Measurement (mm)'] / 1000
  del magnolia_df['Distance to Water Measurement (mm)']
  magnolia_df = magnolia_df.resample('60min').mean()

  magnolia_df = magnolia_df.rolling(13, center=True).mean()

  magnolia_df['TARGET (+' + str(CHANGING_TIME) + ')'] = magnolia_df['Distance to Water Measurement (m)'].shift(-1 * CHANGING_TIME)
  magnolia_df = magnolia_df.dropna()

  return magnolia_df

### **ANN #2**

In [17]:
# Creates second ANN dataframe
def create_second_ann_df(predictions_df, magnolia_df, second_winds_df, CHANGING_TIME):
  second_df = predictions_df[['PredictedPWL']].resample('60min').mean()
  second_df = second_df[magnolia_df.index[0]:magnolia_df.index[-1]]
  second_df['TARGET (+' + str(CHANGING_TIME) + ')'] = magnolia_df['TARGET (+' + str(CHANGING_TIME) + ')']
  second_df = second_df.rename(columns={'PredictedPWL': 'PredictedPWL (+' + str(CHANGING_TIME - 5) + ')'})
  second_df = second_df.dropna()

  organized_df = pd.DataFrame()
  column_names = ['PredictedPWL (+' + str(CHANGING_TIME - 5) + ')']

  for column in column_names:
    for i in range(12, 0, -1):
      if(i % 1 == 0): # Interchangeable
        df_shifted = second_df.shift(1 * i)
        organized_df[column + " -" + str(i)] = df_shifted[column]
    organized_df[column] = second_df[column]


  temp1 = organized_df.dropna().resample('60min').mean()
  temp2 = second_winds_df[organized_df.index[0]:organized_df.index[-1]].resample('60min').mean()
  final_second_df = temp1.set_index(temp1.index).join(temp2.set_index(temp2.index))

  final_second_df['TARGET (+' + str(CHANGING_TIME)  + ')'] = magnolia_df['TARGET (+' + str(CHANGING_TIME)  + ')']
  final_second_df = final_second_df.dropna()

  return final_second_df

In [18]:
def choose_second_ann_model(SHAPE, LAYERS, REGULARIZATION, UNITS1, ACT1, UNITS2, ACT2):
  if(LAYERS == 1):
    if(REGULARIZATION == 1):
      model = tf.keras.Sequential([
        tf.keras.layers.Dense(UNITS1,
                              activation= ACT1,
                              input_shape = SHAPE
                              ),
        tf.keras.layers.Dense(1),
      ])

    elif(REGULARIZATION == 2):
      model = tf.keras.Sequential([
        tf.keras.layers.Dense(UNITS1,
                              activation= ACT1,
                              input_shape = SHAPE,
                              kernel_regularizer = tf.keras.regularizers.L2(l2=0.0001)
                              ),
        tf.keras.layers.Dense(1),
      ])

    elif(REGULARIZATION == 3):
      model = tf.keras.Sequential([
        tf.keras.layers.Dense(UNITS1,
                              activation= ACT1,
                              input_shape = SHAPE,
                              kernel_regularizer = tf.keras.regularizers.L2(l2=0.0001),
                              bias_regularizer = tf.keras.regularizers.L2(l2=0.0001)
                              ),
        tf.keras.layers.Dense(1),
      ])
  if(LAYERS == 2):
    if(REGULARIZATION == 1):
      model = tf.keras.Sequential([
        tf.keras.layers.Dense(UNITS1,
                              activation= ACT1,
                              input_shape = SHAPE
                              ),
        tf.keras.layers.Dense(UNITS2,
                              activation= ACT2,
                              ),
        tf.keras.layers.Dense(1),
      ])
    elif(REGULARIZATION == 2):
      model = tf.keras.Sequential([
        tf.keras.layers.Dense(UNITS1,
                              activation= ACT1,
                              input_shape = SHAPE,
                              kernel_regularizer = tf.keras.regularizers.L2(l2=0.0001)
                              ),
        tf.keras.layers.Dense(UNITS2,
                              activation= ACT2,
                              kernel_regularizer = tf.keras.regularizers.L2(l2=0.0001)
                              ),
        tf.keras.layers.Dense(1),
      ])
    elif(REGULARIZATION == 3):
      model = tf.keras.Sequential([
        tf.keras.layers.Dense(UNITS1,
                              activation= ACT1,
                              input_shape = SHAPE,
                              kernel_regularizer = tf.keras.regularizers.L2(l2=0.0001),
                              bias_regularizer = tf.keras.regularizers.L2(l2=0.0001)
                              ),
        tf.keras.layers.Dense(UNITS2,
                              activation= ACT2,
                              kernel_regularizer = tf.keras.regularizers.L2(l2=0.0001),
                              bias_regularizer = tf.keras.regularizers.L2(l2=0.0001)
                              ),
        tf.keras.layers.Dense(1),
      ])

  return model

In [19]:
def train_second_ann(second_df, random_id, CHANGING_TIME):
  TRAIN_PERCENT = [0, 1, 2, 3, 4, 5, 6]
  VAL_PERCENT = [7, 8]
  TEST_PERCENT = [9]

  # availableBatches = [1, 2, 3, 4]
  availableBatches = [2]
  # availableActivations = ['relu', 'leaky_relu', 'silu', 'tanh', 'softplus', 'elu', 'selu', 'gelu', 'mish']
  availableActivations = ['relu']
  # availableUnits = [1, 2, 4, 8, 16, 32, 64, 128]
  availableUnits = [32, 64, 128]
  # regularizationMethod = [1, 2, 3] # none, kernel, kernel+bias
  regularizationMethod = [2] # none, kernel, kernel+bias
  totalLayers = [1]

  GROUPS_PER_BATCH = 10
  BATCHES = random.choice(availableBatches)
  REGULARIZATION = random.choice(regularizationMethod)
  LAYERS = random.choice(totalLayers)
  ACT1 = random.choice(availableActivations)
  UNITS1 = random.choice(availableUnits[1:])
  ACT2 = random.choice(availableActivations)
  UNITS2 = random.choice(availableUnits[:availableUnits.index(UNITS1)])

  #

  temp_df = second_df.copy()

  remainder = -1 * (len(temp_df) % BATCHES)
  if(remainder != 0):
    temp_df = temp_df[:remainder]
  group_length = len(temp_df) // BATCHES // GROUPS_PER_BATCH

  allMAE = []
  allCF15 = []
  allCF10 = []
  allCF5 = []
  allLD = []

  for i in range(0, 10):
    train_df = temp_df[0:0]
    val_df = temp_df[0:0]
    test_df = temp_df[0:0]

    counter = 0
    mode_counter = i

    while(True):
      if(counter * group_length > len(temp_df)):
        break

      if(mode_counter in TRAIN_PERCENT):
        mode = 0
      elif(mode_counter in VAL_PERCENT):
        mode = 1
      elif(mode_counter in TEST_PERCENT):
        mode = 2

      if(mode == 0):
        train_df = pd.concat([train_df, temp_df[counter * group_length:(counter + 1) * group_length]], ignore_index=False)
      elif(mode == 1):
        val_df = pd.concat([val_df, temp_df[counter * group_length:(counter + 1) * group_length]], ignore_index=False)
      elif(mode == 2):
        test_df = pd.concat([test_df, temp_df[counter * group_length:(counter + 1) * group_length]], ignore_index=False)

      counter += 1
      mode_counter += 1

      if(mode_counter == 10):
        mode = 0
        mode_counter = 0

    X_train = train_df.drop('TARGET (+' + str(CHANGING_TIME)  + ')',axis=1)
    y_train = train_df['TARGET (+' + str(CHANGING_TIME)  + ')']

    X_val = val_df.drop('TARGET (+' + str(CHANGING_TIME)  + ')',axis=1)
    y_val = val_df['TARGET (+' + str(CHANGING_TIME)  + ')']

    X_test = test_df.drop('TARGET (+' + str(CHANGING_TIME)  + ')',axis=1)
    y_test = test_df['TARGET (+' + str(CHANGING_TIME)  + ')']
    SHAPE = [X_train.shape[1]]

    #

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  patience=20,
                                                  mode='min')

    model = choose_second_ann_model(SHAPE, LAYERS, REGULARIZATION, UNITS1, ACT1, UNITS2, ACT2)

    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=[tf.keras.metrics.MeanAbsoluteError()])

    losses = model.fit(X_train, y_train,
                      validation_data=(X_val, y_val),
                      callbacks=[early_stopping],
                      epochs=1000, batch_size = 32, verbose = 0)

    #

    predictions = model.predict(X_test, verbose = 0)
    predictions_df = pd.DataFrame()
    predictions_df.index = X_test.index
    predictions_df['Predictions'] = predictions
    predictions_df['Actual'] = y_test

    fig = go.Figure(data=go.Scatter(x=predictions_df['Actual'].index, y=predictions_df['Actual'], mode='lines', line_color='#000000', name = "Actual"))
    fig.add_trace(go.Scatter(x=predictions_df['Predictions'].index, y=predictions_df['Predictions'], mode='lines', line_color='#FF0000', name = "Predictions"))
    fig.update_traces(connectgaps=False)
    fig.update_layout(title = "")
    fig.update_yaxes(range=[1, 3])
    fig.show()

    #mae
    mae = sum(abs(predictions_df['Predictions'] - predictions_df['Actual'])) / len(predictions_df['Predictions']) * 100
    mae = round(mae, 2)

    # cf 15
    counter15 = 0
    counter10 = 0
    counter5 = 0
    array = abs(predictions_df['Predictions'] - predictions_df['Actual'])
    for j in range(0, len(array)):
      if(array[j] > .15):
        counter15 += 1
      if(array[j] > .1):
        counter10 += 1
      if(array[j] > .05):
        counter5 += 1

    cf15 = (100 - (counter15 / len(array)) * 100)
    cf10 = (100 - (counter10 / len(array)) * 100)
    cf5 = (100 - (counter5 / len(array)) * 100)
    cf5 = round(cf5, 2)

    # largest difference
    ld = -1000
    for j in range(0, len(array)):
      if(array[j] > ld):
        ld = array[j]

    print('MAE:', mae)
    print('CF15:', cf15)
    print('CF10:', cf10)
    print('CF5:', cf5)
    print('LD:', ld)
    if(LAYERS == 1):
      model.save(f'{random_id}___2ndANN_{CHANGING_TIME}hr___{i}_{UNITS1}{ACT1}_{BATCHES}batches_{REGULARIZATION}reg___{mae}mae_{cf5}cf5.keras')
      print(f'{random_id}___2ndANN_{CHANGING_TIME}hr___{i}_{UNITS1}{ACT1}_{BATCHES}batches_{REGULARIZATION}reg___{mae}mae_{cf5}cf5.keras')
    else:
      model.save(f'{random_id}___2ndANN_{CHANGING_TIME}hr___{i}_{UNITS1}{ACT1}_{UNITS2}{ACT2}_{BATCHES}batches_{REGULARIZATION}reg___{mae}mae_{cf5}cf5.keras')
      print(f'{random_id}___2ndANN_{CHANGING_TIME}hr___{i}_{UNITS1}{ACT1}_{UNITS2}{ACT2}_{BATCHES}batches_{REGULARIZATION}reg___{mae}mae_{cf5}cf5.keras')
    print('-' * 100)
    keras.backend.clear_session()

In [20]:
# "????___2ndANN_24hr___64relu_16silu_2batches_2reg___4.23mae_90.43cf15.keras"

### **Main Loop**

In [21]:
# all of this will be in a for loop eventually, with changing_time iterating through different lead times
# CHANGING_TIME = 24
random_id = int(1000000 - float(str(round(time.time()))[4:]))
for CHANGING_TIME in [12, 24, 48]:
  # FIRST ANN
  # ----------------------------------------------------------------------------------------------------
  first_df, first_pwl_conversion, second_winds_df = create_first_ann_df(ports_df, CHANGING_TIME)
  first_X_train, first_y_train, first_X_val, first_y_val, first_X_test, first_y_test = generate_first_ann_split(first_df)
  # plot_first_ann(first_y_train, first_y_val, first_y_test)

  first_model, first_losses = train_first_ann(first_X_train, first_y_train, first_X_val, first_y_val, first_X_test, first_y_test, 256, 'relu')
  first_predictions_df = generate_first_ann_predictions(first_model, first_X_test, first_y_test)
  converted_first_predictions_df = convert_surge_to_pwl(first_predictions_df, first_pwl_conversion)
  # plot_first_ann_predictions(converted_first_predictions_df)

  first_mae, first_cf = evaluate_first_ann(converted_first_predictions_df)

  first_model_name = f'{random_id}_1stANN_{CHANGING_TIME}hr_{first_mae}mae_{first_cf}cf15.keras'
  first_model.save(first_model_name)
  # ----------------------------------------------------------------------------------------------------


  # MAGNOLIA
  # ----------------------------------------------------------------------------------------------------
  MAGNOLIA_DATA_PATH = '/content/Magnolia Data 09_24.csv' #mag
  magnolia_df = create_magnolia_df(MAGNOLIA_DATA_PATH, CHANGING_TIME)[:-100] # to account for not enough data
  # graph_data(magnolia_df)
  # ----------------------------------------------------------------------------------------------------

  # SECOND ANN
  # ----------------------------------------------------------------------------------------------------
  second_df = create_second_ann_df(converted_first_predictions_df, magnolia_df, second_winds_df, CHANGING_TIME)
  train_second_ann(second_df, random_id, CHANGING_TIME)
  # ----------------------------------------------------------------------------------------------------
  for i in range(0, 10):
    print("")

MAE: 4.71
CF15: 99.77728285077951
CF10: 89.64365256124722
CF5: 60.36
LD: 0.16224406888286858
224524___2ndANN_12hr___0_64relu_2batches_2reg___4.71mae_60.36cf5.keras
----------------------------------------------------------------------------------------------------


MAE: 2.94
CF15: 100.0
CF10: 98.7750556792873
CF5: 82.29
LD: 0.12801956219247668
224524___2ndANN_12hr___1_64relu_2batches_2reg___2.94mae_82.29cf5.keras
----------------------------------------------------------------------------------------------------


MAE: 3.22
CF15: 97.99554565701558
CF10: 95.43429844097996
CF5: 82.96
LD: 0.24559125772270796
224524___2ndANN_12hr___2_64relu_2batches_2reg___3.22mae_82.96cf5.keras
----------------------------------------------------------------------------------------------------


MAE: 3.78
CF15: 99.88864142538975
CF10: 96.21380846325167
CF5: 72.05
LD: 0.15274472580660303
224524___2ndANN_12hr___3_64relu_2batches_2reg___3.78mae_72.05cf5.keras
----------------------------------------------------------------------------------------------------


MAE: 3.88
CF15: 97.55011135857461
CF10: 92.87305122494432
CF5: 76.06
LD: 0.2929813039228
224524___2ndANN_12hr___4_64relu_2batches_2reg___3.88mae_76.06cf5.keras
----------------------------------------------------------------------------------------------------


...

MAE: 6.13
CF15: 92.3887587822014
CF10: 82.08430913348946
CF5: 53.04
LD: 0.3502393689674963
224524___2ndANN_48hr___5_128relu_2batches_2reg___6.13mae_53.04cf5.keras
----------------------------------------------------------------------------------------------------


MAE: 7.42
CF15: 87.93911007025761
CF10: 72.95081967213115
CF5: 44.38
LD: 0.3521140570455694
224524___2ndANN_48hr___6_128relu_2batches_2reg___7.42mae_44.38cf5.keras
----------------------------------------------------------------------------------------------------


MAE: 6.71
CF15: 94.6135831381733
CF10: 77.98594847775176
CF5: 40.75
LD: 0.28399667595848666
224524___2ndANN_48hr___7_128relu_2batches_2reg___6.71mae_40.75cf5.keras
----------------------------------------------------------------------------------------------------


MAE: 5.4
CF15: 91.92037470725995
CF10: 83.2552693208431
CF5: 62.06
LD: 0.2560007548100396
224524___2ndANN_48hr___8_128relu_2batches_2reg___5.4mae_62.06cf5.keras
----------------------------------------------------------------------------------------------------


MAE: 5.84
CF15: 96.4367816091954
CF10: 84.59770114942529
CF5: 53.1
LD: 0.4109418855097844
224524___2ndANN_48hr___9_128relu_2batches_2reg___5.84mae_53.1cf5.keras
----------------------------------------------------------------------------------------------------










